In [1]:
import json
from data_adaptor import DataAdaptor

## 2WikiMultihopQA
Authors only use question-answer information. The context is not provided.
> "we use only the question-answer pairs from these datasets, not any passages of relevant text that they contain. These datasets both contain 2-hop compositional questions sourced from facts that appear in Wikipedia articles." - Press, et al.

Authors do not use this dataset to measure compositionality gap which requires known sub-questions and answers to measure.
> "Note that the rest of this section shows that elicitive prompts improve performance but does not show that they narrow the compositionality gap since we lack sub-questions for datasets other than CC." - Press, et al.

In [12]:
from data_loaders import load_2WikiMultihopQA

In [13]:
# Load the training data
wiki_sample = load_2WikiMultihopQA(n_examples=5, split='train')

In [29]:
# Print the first example in pretty format
print(json.dumps(wiki_sample[0], indent=4))

{
    "_id": "13f5ad2c088c11ebbd6fac1f6bf848b6",
    "type": "bridge_comparison",
    "question": "Are director of film Move (1970 Film) and director of film M\u00e9diterran\u00e9e (1963 Film) from the same country?",
    "context": [
        [
            "Stuart Rosenberg",
            [
                "Stuart Rosenberg (August 11, 1927 \u2013 March 15, 2007) was an American film and television director whose motion pictures include \"Cool Hand Luke\" (1967), \"Voyage of the Damned\" (1976), \"The Amityville Horror\" (1979), and \"The Pope of Greenwich Village\" (1984).",
                "He was noted for his work with actor Paul Newman."
            ]
        ],
        [
            "M\u00e9diterran\u00e9e (1963 film)",
            [
                "M\u00e9diterran\u00e9e is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schl\u00f6ndorff.",
                "It was written by Philippe Sollers and produced by Barbet Schroeder, with music 

Let's look at the evidence required to answer the question. It's possible we create prompt examples using these evidences to insert into the fine-tuning set. For example,

**Examplar:**
```
Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg.
Follow up: Who is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no
```

In [6]:
wiki_sample[0]["evidences"]

[['Move (1970 film)', 'director', 'Stuart Rosenberg'],
 ['Méditerranée (1963 film)', 'director', 'Jean-Daniel Pollet'],
 ['Stuart Rosenberg', 'country of citizenship', 'American'],
 ['Jean-Daniel Pollet', 'country of citizenship', 'French']]

### Adapting to Self-Ask Examplar

In [14]:
wiki_adaptor = DataAdaptor(dataset="2WikiMultihopQA")

In [15]:
wiki_examplars = wiki_adaptor.generate_examplars(wiki_sample, strategy="self-ask")
for examplar in wiki_examplars:
    print(examplar)

Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: What is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no

Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?
Are follow up questions needed here: Yes.
Follow up: What is the director of The Falcon (film)?
Intermediate answer: Vatroslav Mimica
Follow up: What is the director of Valentin the Good?
Intermediate answer: Martin Frič
Follow up: What is the country of citizenship of Vatroslav Mimica?
Intermediate answer: Croatian
Fol

### Adapting to Self-Ask Training Example
We can augment the target texts in the dataset with the self-ask rationale to fine-tune a language model to generate text with the self-ask rationale.

In [16]:
wiki_training_examples = wiki_adaptor.generate_training_examples(wiki_sample, strategy="self-ask")
for training_example in wiki_training_examples:
    print(json.dumps(training_example, indent=4))

{
    "prompt": "Question: Are director of film Move (1970 Film) and director of film M\u00e9diterran\u00e9e (1963 Film) from the same country?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: What is the director of Move (1970 film)?\nIntermediate answer: Stuart Rosenberg\nFollow up: What is the director of M\u00e9diterran\u00e9e (1963 film)?\nIntermediate answer: Jean-Daniel Pollet\nFollow up: What is the country of citizenship of Stuart Rosenberg?\nIntermediate answer: American\nFollow up: What is the country of citizenship of Jean-Daniel Pollet?\nIntermediate answer: French\nSo the final answer is: no\n"
}
{
    "prompt": "Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: What is the director of The Falcon (film)?\nIntermediate answer: Vatroslav Mimica\nFollow up: What is the director of Valentin the Good?\nIntermediate answer: M

In [11]:
print(wiki_training_examples[0]["prompt"])
print(wiki_training_examples[0]["target"])

Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here:

Yes.
Follow up: What is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no



### Augment with In-Context Examplars and Self-Ask Rationale Targets
We can combine the two above to create an augmented fine-tuning dataset:
1. Prompt text has in-context examplars
2. Target text has the self-ask rationale

In [17]:
training_examplars = wiki_examplars[:4]
augmented_example = {"prompt" : "\n".join(training_examplars) + "\n" + wiki_training_examples[4]["prompt"], "target" : wiki_training_examples[4]["target"]}
print("--------- Augmented Prompt ---------")
print(augmented_example["prompt"])
print("--------- Target ---------")
print(augmented_example["target"])

--------- Augmented Prompt ---------
Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: What is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no

Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?
Are follow up questions needed here: Yes.
Follow up: What is the director of The Falcon (film)?
Intermediate answer: Vatroslav Mimica
Follow up: What is the director of Valentin the Good?
Intermediate answer: Martin Frič
Follow up: What is the country of citizenship of Vatroslav Mimi

## Compositional Celebrities

In [2]:
from data_loaders import load_CompositionalCelebrities

In [3]:
cc_sample = load_CompositionalCelebrities(n_examples=5, split='train')

/Users/richardmathews/Downloads/W266 project/data_loaders.py:54: UserWarning: There is only one file. The data has not been splitted yet.
  warnings.warn("There is only one file. The data has not been splitted yet.")


Let's take a look at an example. We see the subquestions are clearly delineated, with answers, as well as the compositional question itself. The other metadata is not necessary for our research.

In [4]:
# Print the first example in pretty format
print(json.dumps(cc_sample[0], indent=4))

{
    "Q1": "What is the birthplace (country only) of Rumi?",
    "A1": [
        "Afghanistan"
    ],
    "Q2": "What is the capital of Afghanistan?",
    "A2": [
        "Kabul"
    ],
    "Question": "What is the capital of the birthplace of Rumi?",
    "Answer": [
        "Kabul"
    ],
    "birth_country": "Afghanistan",
    "capital": [
        "Kabul"
    ],
    "person": "Rumi",
    "category": "birthplace_capital",
    "person_id": 0
}


### Adapting to Self-Ask Examplar
It's quite simple to restructure this as a "self-ask" examplar for prompting/fine-tuning.

In [5]:
cc_adaptor = DataAdaptor(dataset="CompositionalCelebrities")

In [6]:
cc_examplars = cc_adaptor.generate_examplars(cc_sample, strategy="self-ask")
for examplar in cc_examplars:
    print(examplar)

Question: What is the capital of the birthplace of Rumi?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Rumi?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capital of the birthplace of Ahmad Shah Massoud?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Ahmad Shah Massoud?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capital of the birthplace of Mahmud of Ghazni?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Mahmud of Ghazni?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capital of the birthplace of Ahmad Shah D

### Adapting to Self-Ask Training Example
We can augment the target texts in the dataset with the self-ask rationale to fine-tune a language model to generate text with the self-ask rationale.

In [7]:
cc_training_examples = cc_adaptor.generate_training_examples(cc_sample, strategy="self-ask")
for training_example in cc_training_examples:
    print(json.dumps(training_example, indent=4))

{
    "prompt": "Question: What is the capital of the birthplace of Rumi?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: What is the birthplace (country only) of Rumi?\nIntermediate answer: Afghanistan\nFollow up: What is the capital of Afghanistan?\nIntermediate answer: Kabul\nSo the final answer is: Kabul\n"
}
{
    "prompt": "Question: What is the capital of the birthplace of Ahmad Shah Massoud?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: What is the birthplace (country only) of Ahmad Shah Massoud?\nIntermediate answer: Afghanistan\nFollow up: What is the capital of Afghanistan?\nIntermediate answer: Kabul\nSo the final answer is: Kabul\n"
}
{
    "prompt": "Question: What is the capital of the birthplace of Mahmud of Ghazni?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: What is the birthplace (country only) of Mahmud of Ghazni?\nIntermediate answer: Afghanistan\nFollow up: What is the capital of Af

In [8]:
print(cc_training_examples[0]["prompt"])
print(cc_training_examples[0]["target"])

Question: What is the capital of the birthplace of Rumi?
Are follow up questions needed here:

Yes.
Follow up: What is the birthplace (country only) of Rumi?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul



### Augment with In-Context Examplars and Self-Ask Rationale Targets
We can combine the two above to create an augmented fine-tuning dataset:
1. Prompt text has in-context examplars
2. Target text has the self-ask rationale

In [10]:
training_examplars = cc_examplars[:4]
augmented_example = {"prompt" : "\n".join(training_examplars) + "\n" + cc_training_examples[4]["prompt"], "target" : cc_training_examples[4]["target"]}
print("--------- Augmented Prompt ---------")
print(augmented_example["prompt"])
print("--------- Target ---------")
print(augmented_example["target"])

--------- Augmented Prompt ---------
Question: What is the capital of the birthplace of Rumi?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Rumi?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capital of the birthplace of Ahmad Shah Massoud?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Ahmad Shah Massoud?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capital of the birthplace of Mahmud of Ghazni?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Mahmud of Ghazni?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capi